In [ ]:
import torch
!pip install transformers
from transformers import AutoModel, AutoTokenizer 

bertweet = AutoModel.from_pretrained("vinai/bertweet-large")

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")

# INPUT TWEET IS ALREADY NORMALIZED!
line = "DHEC confirms HTTPURL via @USER :crying_face:"

input_ids = torch.tensor([tokenizer.encode(line)])

with torch.no_grad():
    features = bertweet(input_ids)  # Models outputs are now tuples
    
## With TensorFlow 2.0+:
# from transformers import TFAutoModel
# bertweet = TFAutoModel.from_pretrained("vinai/bertweet-large")

In [2]:
!pip install torch
!pip install fairseq
!pip install fastBPE

     |████████████████████████████████| 1.7 MB 4.8 MB/s 
     |████████████████████████████████| 90 kB 7.6 MB/s 
     |████████████████████████████████| 145 kB 29.6 MB/s 
     |████████████████████████████████| 74 kB 2.0 MB/s 
     |████████████████████████████████| 112 kB 26.2 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=3b9327acc02c8f7bf4c6fb2efa1e129a77239489851b83731269f8721cc1a972
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime


  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=483174 sha256=b3c05077186922d2bf9503f5b503967f9b44d565fe89faf54e9d36697b89a12e
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
Successfully built fastBPE


In [3]:
!wget https://public.vinai.io/BERTweet_base_fairseq.tar.gz

--2022-02-23 01:19:26--  https://public.vinai.io/BERTweet_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 13.226.50.34, 13.226.50.128, 13.226.50.14, ...
Connecting to public.vinai.io (public.vinai.io)|13.226.50.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1237836670 (1.2G) [application/x-tar]
Saving to: ‘BERTweet_base_fairseq.tar.gz’

BERTweet_base_fairs 100%[===================>]   1.15G  71.0MB/s    in 17s     

2022-02-23 01:19:44 (71.5 MB/s) - ‘BERTweet_base_fairseq.tar.gz’ saved [1237836670/1237836670]



In [4]:
!ls

BERTweet_base_fairseq.tar.gz  sample_data


In [5]:
import torch

# Load BERTweet-base in fairseq
from fairseq.models.roberta import RobertaModel
BERTweet = RobertaModel.from_pretrained('/content/BERTweet_base_fairseq.tar.gz', checkpoint_file='model.pt')
BERTweet.eval()  # disable dropout (or leave in train mode to finetune)

# Incorporate the BPE encoder into BERTweet-base 
from fairseq.data.encoders.fastbpe import fastBPE  
from fairseq import options  
parser = options.get_preprocessing_parser()  
parser.add_argument('--bpe-codes', type=str, help='path to fastBPE BPE', default="/content/BERTweet_base_fairseq.tar.gz/bpe.codes")  
args, unknown = parser.parse_known_args()  
BERTweet.bpe = fastBPE(args) #Incorporate the BPE encoder into BERTweet

# INPUT TEXT IS TOKENIZED!
line = "SC has first two presumptive cases of coronavirus , DHEC confirms HTTPURL via @USER :cry:"  

# Extract the last layer's features  
subwords = BERTweet.encode(line)  
last_layer_features = BERTweet.extract_features(subwords)  
assert last_layer_features.size() == torch.Size([1, 21, 768])  
  
# Extract all layer's features (layer 0 is the embedding layer)  
all_layers = BERTweet.extract_features(subwords, return_all_hiddens=True)  
assert len(all_layers) == 13  
assert torch.all(all_layers[-1] == last_layer_features)  

# Filling marks  
masked_line = 'SC has first two presumptive cases of  <mask> , DHEC confirms HTTPURL via @USER :cry:'  
topk_filled_outputs = BERTweet.fill_mask(masked_line, topk=5)  
for candidate in topk_filled_outputs:  
    print(candidate)
    # ('SC has first two presumptive cases of coronavirus , DHEC confirms HTTPURL via @USER :cry:', 0.8643638491630554, 'coronavirus')
    # ('SC has first two presumptive cases of Coronavirus , DHEC confirms HTTPURL via @USER :cry:', 0.04520644247531891, 'Coronavirus')
    # ('SC has first two presumptive cases of #coronavirus , DHEC confirms HTTPURL via @USER :cry:', 0.035870883613824844, '#coronavirus')
    # ('SC has first two presumptive cases of #COVID19 , DHEC confirms HTTPURL via @USER :cry:', 0.029708299785852432, '#COVID19')
    # ('SC has first two presumptive cases of #Coronavirus , DHEC confirms HTTPURL via @USER :cry:', 0.005226477049291134, '#Coronavirus')



1042301B [00:00, 1646747.36B/s]
456318B [00:00, 1043846.51B/s]


('SC has first two presumptive cases of coronavirus , DHEC confirms HTTPURL via @USER :cry:', 0.864362359046936, 'coronavirus')
('SC has first two presumptive cases of Coronavirus , DHEC confirms HTTPURL via @USER :cry:', 0.0452067106962204, 'Coronavirus')
('SC has first two presumptive cases of #coronavirus , DHEC confirms HTTPURL via @USER :cry:', 0.035871367901563644, '#coronavirus')
('SC has first two presumptive cases of #COVID19 , DHEC confirms HTTPURL via @USER :cry:', 0.029708873480558395, '#COVID19')
('SC has first two presumptive cases of #Coronavirus , DHEC confirms HTTPURL via @USER :cry:', 0.005226513370871544, '#Coronavirus')


In [6]:
features

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[ 0.0862,  0.0521, -0.1026,  ...,  0.0147,  0.1891, -0.0736],
                                                        [ 0.1007, -0.0159, -0.6580,  ...,  0.1054,  0.2050,  0.0699],
                                                        [ 0.2216,  0.0395, -0.5482,  ..., -0.1893,  0.3768,  0.2154],
                                                        ...,
                                                        [ 0.0523,  0.2308, -0.8912,  ...,  0.3122, -0.0351, -0.0617],
                                                        [-0.0252, -0.1803, -0.0527,  ...,  0.3899, -0.1398, -0.2169],
                                                        [ 0.0499,  0.0034, -0.1559,  ..., -0.0495,  0.1139, -0.0543]]])),
                                              ('pooler_output',
                                               tensor([[-0.1783,  0.2051, -0.5331,  ...,  0.454

In [7]:
import pandas as pd

In [8]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
df = pd.read_csv('/content/gdrive/MyDrive/covid19_tweets.csv')

In [10]:
df=df.head(n=16384)

In [11]:
feature_list_of_lists = []

features['last_hidden_state'].reshape(1,-1)[0].tolist()#feature set for 1 tweet

feature_list_of_lists.append(features['last_hidden_state'].reshape(1,-1)[0].tolist())

In [16]:
df_features = pd.DataFrame(feature_list_of_lists)

In [28]:
df_features = df_features.transpose() #xvalues: embeddings from BERTweets model , use these to train new model on similar but different task
#BERTweets tasks were POS tagging, NER, and text classification
#my model is for prediction of whether tweet 

In [75]:
df_features=df_features.astype(float)

In [41]:
df_verified = df['user_verified'] #y values to predict

In [71]:
df_target = pd.DataFrame(df_verified)

In [101]:
df_target

,user_verified
0,False
1,True
2,False
3,False
4,False
...,...
16379,False
16380,False
16381,False
16382,False


In [102]:
X = df_features
y = df_target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)

In [103]:
y_train

,user_verified
1021,False
16233,False
13647,False
3188,False
7753,True
...,...
7565,False
7356,False
5551,False
12458,False


In [104]:
X_train=X_train.astype(float)

In [106]:
y_train=y_train.astype(int)

In [107]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [108]:
y_predictions = clf.predict(X_test)

In [115]:
y_predictions

array([0, 0, 0, ..., 0, 0, 1])

In [119]:
y_test=y_test.astype(int)

In [120]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predictions)

0.7628928898382668